#### 2D Visualization of 3D IMS Cell Tracking Data

- Author: Zachary Neronha (Wong Lab)
- Created: 26 Oct, 2018
- Last Modified: Nov 21 (Dhananjay Bhaskar)

##### Import packages and create helper functions

In [4]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
from __future__ import division
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='zjneronha', api_key='PMrKGUGAet6kUZ5Rcf5E')
import brewer2mpl as cbrew

In [3]:
# function 1: return the cells within a certain location
def findRange(xPts,yPts,xRange,yRange):
    cellList = []
    for ii in range(0,np.size(xPts,0)):
        qLog = np.greater(xPts[ii,:],xRange[0]) & np.less(xPts[ii,:],xRange[1])
        pLog = np.greater(yPts[ii,:],yRange[0]) & np.less(yPts[ii,:],yRange[1])
        qLog = sum(qLog)
        pLog = sum(pLog)

        if qLog > 0 and pLog >0:
            cellList.append(ii)
    
    return cellList

# function 2: pull and export only the cells of interest 
def selExport(xPts,yPts,zPts,MyCells,pathname):
    selX = xPts[MyCells,:]
    selY = yPts[MyCells,:]
    selZ = zPts[MyCells,:]
    
    ptsSZE = sum(sum(np.logical_not(np.isnan(selX))))
    selPTS = np.empty((ptsSZE,3))
    loopcounter = 0
    
    #convert back to direct matrix format and export
    for ii in range(0,np.size(selX,0)):
        for jj in range(0,np.size(selX,1)):
            if np.isnan(selX[ii,jj])==0:
                selPTS[loopcounter,:] = [selX[ii,jj],selY[ii,jj],selZ[ii,jj]]
                loopcounter = loopcounter + 1
                
    dTPS = pd.DataFrame(selPTS)
    dTPS.to_csv(pathname,index=False,header = False)
    
    print("Feature Complete; Cluster AllTemp Pts File Saved!")

# function 3: pull out and export all points at one time frame
def temExport(xPts,yPts,zPts,Frame,pathname):
    selX = xPts[:,Frame]
    selY = yPts[:,Frame]
    selZ = zPts[:,Frame]
    
    ptsSZE = sum(np.logical_not(np.isnan(selX)))
    selPTS = np.empty((ptsSZE,3))
    loopcounter = 0
    
    #convert back to direct matrix format and export
    for ii in range(0,np.size(selX,0)):
        if np.isnan(selX[ii])==0:
            selPTS[loopcounter,:] = [selX[ii],selY[ii],selZ[ii]]
            loopcounter = loopcounter + 1
                
    dTPS = pd.DataFrame(selPTS)
    dTPS.to_csv(pathname,index=False,header = False)
    
    print("Feature Complete; Cluster SingTemp Pts File Saved!")

#### Import position data, calculate velocity data

In [6]:
wname = "A07"
interval = (1/3) #hours between imaging 

# Import data from Excel file
fname = os.sep.join(["Wk2_IMS_Crop_Tracks", 
                     "3d sc xmas 9-20-18 week 2_"+wname+"_spots17_Statistics", 
                     wname+"_combineddata.xls"])

xlsZ = pd.ExcelFile(fname)

dxx = pd.read_excel(xlsZ,'xStore')
dyy = pd.read_excel(xlsZ,'yStore')
dzz = pd.read_excel(xlsZ,'zStore')
xDat = np.array(dxx)
yDat = np.array(dyy)
zDat = np.array(dzz)

#Can restict timeframes to some extent
#xDat = xDat[:,(24*3):(60*3)]
#yDat = yDat[:,(24*3):(60*3)]
#zDat = zDat[:,(24*3):(60*3)]

xVel = np.empty((np.size(xDat,0),np.size(xDat,1)))
yVel = np.empty((np.size(xDat,0),np.size(xDat,1)))
zVel = np.empty((np.size(xDat,0),np.size(xDat,1)))
xVel[:] = np.nan
yVel[:] = np.nan
zVel[:] = np.nan

for k in range(0,np.size(xDat,1)-1):
    xVel[:,k] = xDat[:,k+1]-xDat[:,k]
    yVel[:,k] = yDat[:,k+1]-yDat[:,k]
    zVel[:,k] = zDat[:,k+1]-zDat[:,k]

### Plot the data temporally
- Note each color corresponds to a distinct cell track over the time window

In [7]:
pltdata = []
for i in range(0,np.size(xDat,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = xDat[i,:],
        y = yDat[i,:],   
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='basic-line1')

In [8]:
xRan = [540,575]
yRan = [330,370]
celL = findRange(xDat,yDat,xRan,yRan)
print(celL)

[]


In [9]:
#X and Y temporal plots, frequency estimation, etc.
xred = xDat[celL,:]
yred = yDat[celL,:]
trL = np.multiply(xred,0)+1
for k in range(0,np.size(trL,1)):
    trL[:,k] = np.multiply(trL[:,k],k)

trL = np.multiply(trL,interval)

In [10]:
#plot sequentially
pltdata = []
for i in range(0,np.size(xred,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = trL[i,:],
        y = xred[i,:], 
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='XFREQ')

PlotlyEmptyDataError: Empty data list found. Make sure that you populated the list of data objects you're sending and try again.
Questions? Visit support.plot.ly

In [80]:
#plot Y frequency
pltdata = []
for i in range(0,np.size(xred,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = trL[i,:],
        y = yred[i,:], 
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='YFREQ')

In [62]:
pathnameS = "SelectedFeatures/Circuit1_w"+repr(currentwell)+".csv"
selExport(xDat,yDat,zDat,celL,pathnameS)

Feature Complete; File Saved!


#### Breakup analysis for all points in one frame
- Look specifically at plotting
- Then export to a csv for TDA in R

In [15]:
#define frame of interest
FOI = 1

plt_data = []
trace = go.Scatter3d(
            name = 'SyntheticSphere'+repr(i),
            x = xDat[:,FOI],
            y = yDat[:,FOI],
            z = zDat[:,FOI],
            mode = 'markers',
            marker = dict(
                color = 1,
                size = 8,
                symbol = 'circle',
                line = dict(
                    color = 'rgb(180, 180, 180)',
                    width = 1.0
                ),
                opacity = 0.4
            )
        )

plt_data.append(trace)

layout = go.Layout(margin = dict(l = 0, r = 0, b = 0, t = 0))

fig = go.Figure(data=plt_data, layout=layout)

iplot(fig)

In [43]:
pathnameF = "SelectedFeatures/TemBreak_w"+repr(currentwell)+"t"+repr(FOI)+".csv"
temExport(xDat,yDat,zDat,FOI,pathnameF) 

Feature Complete; Cluster SingTemp Pts File Saved!


#### Save frame 1 positions to CSV file

In [20]:
import glob
import os
readpath = glob.glob("Wk2_IMS_Crop_Tracks/*/*.xls")

for path in readpath:
    
    print(path)
    filename = os.path.basename(path)
    wellname = filename.split("_")[0]
    #import data from csv
    xlsZ = pd.ExcelFile(path,engine='xlrd')
    dxx = pd.read_excel(xlsZ,'xStore')
    dyy = pd.read_excel(xlsZ,'yStore')
    dzz = pd.read_excel(xlsZ,'zStore')
    xDat = np.array(dxx)
    yDat = np.array(dyy)
    zDat = np.array(dzz)
    
    ofname = os.sep.join([os.path.dirname(path), wellname+".csv"])
    
    temExport(xDat, yDat, zDat, 1, ofname)

Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_B10_spot11_Statistics/B10_combineddata.xls
Feature Complete; Cluster SingTemp Pts File Saved!
Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A12_spots15_Statistics/A12_combineddata.xls
Feature Complete; Cluster SingTemp Pts File Saved!
Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A07_spots17_Statistics/A07_combineddata.xls
Feature Complete; Cluster SingTemp Pts File Saved!
Wk2_IMS_Crop_Tracks/3d sc xmas 9-20-18 week 2_A08_spots2_Statistics/A08_combineddata.xls
Feature Complete; Cluster SingTemp Pts File Saved!
